In [1]:
pip install pandas matplotlib seaborn

In [2]:

"""
Processamento ETL para dados de telecomunicações - TelecomX_Data.json
"""

# ============================
# 0. IMPORTAÇÃO DE BIBLIOTECAS
# ============================
import os                     # Para manipulação de arquivos e pastas
import json                   # Para leitura de arquivos JSON
import pandas as pd           # Para manipulação de DataFrames
import matplotlib.pyplot as plt  # Para visualizações estáticas
import seaborn as sns            # Para visualizações avançadas
from datetime import datetime    # Para manipulação de datas

# Configurações iniciais
pd.set_option('display.max_columns', None)  # Mostra todas as colunas no print
sns.set_style("whitegrid")                  # Estilo padrão Seaborn
sns.set_palette("pastel")                   # Paleta de cores

# ============================
# 1. EXTRAÇÃO DOS DADOS
# ============================
def carregar_dados_json(caminho_arquivo):
    """
    Função para ler dados de um arquivo JSON e converter em DataFrame
    """
    try:
        # Abrir arquivo JSON
        with open(caminho_arquivo, 'r', encoding='utf-8') as file:
            dados = json.load(file)

        # Verifica o tipo de JSON e cria DataFrame
        if isinstance(dados, list):
            df = pd.DataFrame(dados)
        elif isinstance(dados, dict):
            if any(isinstance(v, dict) for v in dados.values()):
                df = pd.json_normalize(dados)  # Achata o JSON aninhado
            else:
                df = pd.DataFrame.from_dict(dados, orient='index').reset_index()
        else:
            raise ValueError("Formato JSON não reconhecido")

        # Mostrar informações iniciais
        print(f"✅ Dados carregados com sucesso. Total de registros: {len(df)}")
        print("\n🔍 Amostra dos dados (5 primeiras linhas):")
        print(df.head())
        print("\n📋 Estrutura do DataFrame:")
        print(df.info())

        return df

    except FileNotFoundError:
        raise FileNotFoundError("❌ Arquivo não encontrado")
    except json.JSONDecodeError:
        raise ValueError("❌ Erro ao decodificar o arquivo JSON")
    except Exception as e:
        raise Exception(f"❌ Erro inesperado: {str(e)}")

# Chama a função para carregar dados
try:
    df = carregar_dados_json('TelecomX_Data.json')
except Exception as e:
    print(e)
    exit()

# ============================
# 2. TRANSFORMAÇÃO E LIMPEZA
# ============================
def transformar_dados(df):
    """
    Limpeza e transformação dos dados
    """
    # Salva informações iniciais para comparação
    original_shape = df.shape
    original_nulos = df.isnull().sum().sum()

    # Mostra análise inicial de valores nulos e tipos
    print("\n🔍 Análise inicial de qualidade:")
    print("Valores nulos por coluna:")
    print(df.isnull().sum())
    print("\nTipos de dados atuais:")
    print(df.dtypes)

    # Define transformações que serão aplicadas
    transformacoes = {
        'para_datetime': ['data', 'data_inicio', 'data_fim', 'dt_contrato'],
        'para_numerico': ['valor', 'duracao', 'franquia', 'consumo'],
        'para_categoria': ['plano', 'status', 'tipo_servico', 'regiao'],
        'para_texto': ['cliente', 'endereco', 'cidade', 'uf'],
        'remover_colunas': ['codigo_antigo', 'observacoes_internas']
    }

    try:
        # 1️⃣ Remover colunas desnecessárias
        cols_remover = [col for col in transformacoes['remover_colunas'] if col in df.columns]
        df.drop(columns=cols_remover, inplace=True, errors='ignore')

        # 2️⃣ Converter colunas para datetime
        for col in transformacoes['para_datetime']:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')

        # 3️⃣ Converter colunas para numérico
        for col in transformacoes['para_numerico']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # 4️⃣ Converter colunas para categoria
        for col in transformacoes['para_categoria']:
            if col in df.columns:
                df[col] = df[col].astype('category')

        # 5️⃣ Padronizar textos (maiúsculas e remove espaços)
        for col in transformacoes['para_texto']:
            if col in df.columns:
                df[col] = df[col].astype(str).str.upper().str.strip()

        # 6️⃣ Criar colunas derivadas
        if 'data' in df.columns:
            df['ano_mes'] = df['data'].dt.to_period('M')

        if all(col in df.columns for col in ['data_fim', 'data_inicio']):
            df['dias_contrato'] = (df['data_fim'] - df['data_inicio']).dt.days

        if 'duracao' in df.columns:
            df['duracao_horas'] = df['duracao'] / 60

        # Resumo das transformações
        print("\n✅ Transformações aplicadas com sucesso")
        print(f"\n📊 Resumo das transformações:")
        print(f"- Colunas removidas: {len(cols_remover)}")
        print(f"- Registros antes: {original_shape[0]}, Colunas antes: {original_shape[1]}")
        print(f"- Registros após: {df.shape[0]}, Colunas após: {df.shape[1]}")
        print(f"- Valores nulos antes: {original_nulos}, Valores nulos após: {df.isnull().sum().sum()}")

        return df

    except Exception as e:
        print(f"\n❌ Erro durante as transformações: {str(e)}")
        return df

df = transformar_dados(df)

# ============================
# 3. SALVAR E ANALISAR DADOS
# ============================
def salvar_analisar_dados(df):
    """
    Salva os dados tratados em CSV e Parquet e realiza análise exploratória
    """
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        os.makedirs("output", exist_ok=True)

        # Salvar dados
        df.to_csv(f'output/TelecomX_Data_Tratado_{timestamp}.csv', index=False, encoding='utf-8-sig')
        df.to_parquet(f'output/TelecomX_Data_Tratado_{timestamp}.parquet', index=False)
        print(f"\n💾 Dados salvos com sucesso: CSV e Parquet na pasta 'output'")

        # Estatísticas descritivas
        print("\n📈 Estatísticas descritivas:")
        print(df.describe(include='all', datetime_is_numeric=True))

        # Distribuição de categorias
        categorical_cols = [col for col in df.columns if df[col].dtype.name in ['category', 'object']]
        for col in categorical_cols:
            print(f"\n🔠 {col}:")
            print(df[col].value_counts(normalize=True).head(10))

        # Análise temporal
        if 'data' in df.columns:
            print("\n📅 Análise temporal:")
            registros_por_mes = df.groupby(df['data'].dt.to_period('M')).size()
            print(registros_por_mes)

    except Exception as e:
        print(f"\n❌ Erro ao salvar/analisar dados: {str(e)}")

salvar_analisar_dados(df)

# ============================
# 4. RELATÓRIO FINAL E VISUALIZAÇÕES
# ============================
def gerar_relatorio(df):
    """
    Gera gráficos e relatório de qualidade dos dados
    """
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_dir = f'output/relatorio_{timestamp}'
        os.makedirs(report_dir, exist_ok=True)

        # Gráficos para colunas numéricas
        numeric_cols = df.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            plt.figure(figsize=(10, 5))
            sns.histplot(df[col], kde=True)
            plt.title(f'Distribuição de {col}')
            plt.savefig(f'{report_dir}/distribuicao_{col}.png', bbox_inches='tight')
            plt.close()

        # Gráficos para colunas categóricas
        categorical_cols = [col for col in df.columns if df[col].dtype.name in ['category', 'object']]
        for col in categorical_cols:
            plt.figure(figsize=(10, 5))
            df[col].value_counts().head(10).plot(kind='bar')
            plt.title(f'Top 10 - {col}')
            plt.xticks(rotation=45)
            plt.savefig(f'{report_dir}/top10_{col}.png', bbox_inches='tight')
            plt.close()

            if df[col].nunique() <= 5:
                plt.figure(figsize=(8, 8))
                df[col].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=90)
                plt.title(f'Distribuição - {col}')
                plt.savefig(f'{report_dir}/pie_{col}.png', bbox_inches='tight')
                plt.close()

        # Tendência temporal
        if 'data' in df.columns:
            registros_por_mes = df.groupby(df['data'].dt.to_period('M')).size()
            plt.figure(figsize=(12, 6))
            registros_por_mes.plot(kind='line', marker='o')
            plt.title('Tendência Mensal de Registros')
            plt.xlabel('Mês')
            plt.ylabel('Quantidade')
            plt.grid(True)
            plt.savefig(f'{report_dir}/tendencia_mensal.png', bbox_inches='tight')
            plt.close()

        # Relatório de qualidade
        with open(f'{report_dir}/relatorio_qualidade.txt', 'w', encoding='utf-8') as f:
            f.write("RELATÓRIO DE QUALIDADE DOS DADOS - TELECOMX\n")
            f.write("="*50 + "\n\n")
            f.write(f"Data do relatório: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
            f.write(f"Total de registros: {len(df):,}\n")
            f.write(f"Total de colunas: {len(df.columns)}\n\n")
            f.write("RESUMO DE VALORES NULOS:\n")
            f.write(df.isnull().sum().to_string())
            f.write("\n\nTIPOS DE DADOS:\n")
            f.write(df.dtypes.to_string())
            f.write("\n\nESTATÍSTICAS DESCRITIVAS:\n")
            f.write(df.describe(include='all', datetime_is_numeric=True).to_string())

        print(f"\n✅ Relatório gerado com sucesso na pasta: {report_dir}")

    except Exception as e:
        print(f"\n❌ Erro ao gerar relatório: {str(e)}")

gerar_relatorio(df)

print("\n" + "="*50)
print("PROCESSO ETL CONCLUÍDO COM SUCESSO!")
print("="*50)


✅ Dados carregados com sucesso. Total de registros: 7267

🔍 Amostra dos dados (5 primeiras linhas):
   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'Male', 'SeniorCitizen': 1, 'Partne...   
4  0013-EXCHZ   Yes  {'gender': 'Female', 'SeniorCitizen': 1, 'Part...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
3   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
4   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetServi